In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [2]:
# Custom dataset class
class CustomLogoDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for label, subdir in enumerate(['Fake', 'Genuine']):
            folder_path = os.path.join(self.root_dir, subdir)
            for filename in os.listdir(folder_path):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    self.images.append(os.path.join(folder_path, filename))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


In [3]:
# Image transformations
image_transforms = transforms.Compose([
    transforms.Resize((299, 299)),  # InceptionV3 input size
    transforms.ToTensor(),
])

# Prepare the datasets and dataloaders
train_dataset = CustomLogoDataset('FakeReal Logo Detection dataset/train', transform=image_transforms)
test_dataset = CustomLogoDataset('FakeReal Logo Detection dataset/test', transform=image_transforms)

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# Model setup
model = models.inception_v3(pretrained=True)
num_classes = 2
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, num_classes)

# Define the loss function and optimizer
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

d:\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\djiva/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [02:43<00:00, 665kB/s] 


In [7]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs, aux_outputs = model(images)
        loss = criterion(outputs, labels)  # Use outputs instead of model(images)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}")

Epoch 1/10, Loss: 0.335844351422219
Epoch 2/10, Loss: 0.22847873398235866
Epoch 3/10, Loss: 0.1353642942295188
Epoch 4/10, Loss: 0.08991306962534076
Epoch 5/10, Loss: 0.04258652217686176
Epoch 6/10, Loss: 0.050021365839278416
Epoch 7/10, Loss: 0.05550197644957474
Epoch 8/10, Loss: 0.04038353792635635
Epoch 9/10, Loss: 0.04452344152100739
Epoch 10/10, Loss: 0.017877268574444487


In [8]:
# Evaluation
model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [9]:
print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Accuracy of the network on the test images: 99.39393939393939%


In [10]:
# #Save the model
torch.save(model.state_dict(), 'inception_fake_real_logo_detection_model_second.pth')